In [ ]:
## Benchpress Hackathon

The challenge comes with a Jupyter notebook for your implementation and various utilities.
We provide a development set and a validation set you can use to develop your solution.
The development set is for testing your code and consists of 300 problems with a varying number of test cases.
You are free to use all data provided with a problem, a sample has the following structure:

```python
{
    # Unique identifier for the problem in the APPS dataset.
    "problem_id": 4424,
    # The problem statement
    "question": "Given three integers ...",
    # The expected function name and the input/output examples
    # representing test cases.
    "input_output": {
        "fn_name": "expression_matter",
        "inputs": [ ... ],
        "outputs": [ ... ]
    },
    "url": "https://www.codewars.com/kata/5ae62fcf252e66d44d00008e",
    "difficulty": "introductory",
    # The starter code for the problem.
    "starter_code": "def expression_matter(a, b, c):\n\t"
}
```

The validation set is consists of 200 problems, and includes an additional key `test_cases` which is used to score your solution with the provided scoring function.

```python
{
    ...
    "test_cases": {
        "fn_name": "expression_matter",
        "inputs": [ ... ],
        "outputs": [ ... ]
    },
    ...
}
```

### Loading Problems

Use the `load_sample` function to load a problem from the development or validation set.

```python
from utilities import load_sample

problem = load_sample(index=0, dataset_path="./data/dev")
```

### Generating Code

Use the `aleph_alpha_client` to generate code.
Make sure your `AA_TOKEN` is set.

```python
from aleph_alpha_client import Client, CompletionRequest, Prompt

client = Client(AA_TOKEN)

request = CompletionRequest(
    prompt=Prompt.from_text("Your prompt."),
    maximum_tokens=256,
)

# API reference for the client:
# https://aleph-alpha-client.readthedocs.io/en/latest/
response = client.complete(request, model=MODEL)
```

### Running Tests

Use the `run_test_cases` function to run the generated code against the test cases.
The function returns a dictionary with the test results, including the expected output, the generated output, a boolean indicating whether the test passed and a traceback in case of an error.

```python
from utilities import run_test_cases

test_results = run_test_cases(
    problem=problem, 
    generation=response.completions[0].completion, 
    timeout=10,
)
```

### Scoring

Use the `score` function to score your solution on the validation set.
It expects a function that takes a problem and a client and returns a generation.

```python
from utilities import score

passed_problems, passed_test_cases = score(
    generation_func=generate_code, 
    client=client,
    dataset_path="./data/val", 
    length=50,
)
```

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os

AA_TOKEN = ""
# MODEL = "llama-3.1-8b-instruct-long-context"
MODEL = "llama-3.1-70b-instruct-long-context"

if AA_TOKEN is None:
    raise ValueError("Aleph Alpha Playground token is not set.")


In [ ]:
context_json = {
  "0": "def expression_matter(a, b, c):\n    return max(a * (b + c), a + b * c, (a + b) * c, a + b + c)\n",
  "1": "def kooka_counter(laughing):\n    laughing = laughing.replace('hahaha', 'hahaha ').replace('HaHaHa', 'HaHaHa ').strip()\n    return len(laughing.split()) // 2 + len(laughing.split()) % 2\n",
  "2": "def rps(p1, p2):\n    if p1 == p2:\n        return \"Draw!\"\n    if (p1 == \"rock\" and p2 == \"scissors\") or (p1 == \"scissors\" and p2 == \"paper\") or (p1 == \"paper\" and p2 == \"rock\"):\n        return \"Player 1 won!\"\n    return \"Player 2 won!\"\n",
  "3": 'def sum_of_threes(n):\n    if n == 2:\n        return "Impossible"\n    result = []\n    i = 0\n    while n > 0:\n        if n % 3 == 1:\n            result.append(f"3^{i}")\n        n //= 3\n        i += 1\n    return "+".join(reversed(result))\n',
  "4": "def find_average(nums):\n    if len(nums) == 0:\n      return 0\n    return sum(nums) / len(nums)\n",
  '5': 'def merge(line):\n    line = [x for x in line if x != 0]\n    \n    merged = []\n    \n    i = 0\n    \n    while i < len(line):\n        if i < len(line) - 1 and line[i] == line[i + 1]:\n            merged.append(line[i] * 2)\n            i += 2\n        else:\n            merged.append(line[i])\n            i += 1\n    \n    merged += [0] * (len(line) - len(merged)+1)\n    return merged\n',

  "7": "def compare(a, b):\n    def count_selectors(selector):\n        id_count = selector.count('#')\n        class_count = selector.count('.')\n        tag_count = len(selector.replace('#', '').replace('.', '').split()) - 1\n        return (id_count, class_count, tag_count)\n\n    a_count = count_selectors(a)\n    b_count = count_selectors(b)\n\n    if a_count > b_count:\n        return a\n    elif a_count < b_count:\n        return b\n    else:\n        return b\n",
}


In [ ]:
from aleph_alpha_client import Client, CompletionRequest, Prompt
from utilities import load_sample, run_test_cases


client = Client(AA_TOKEN)


def create_context_prompt(sample_size, dataset_path):
    context = []
    indices = list(range(sample_size))
    for index in indices:
        current_context = "**FEW-SHOT EXAMPLE**:\n"
        problem = load_sample(index=index, dataset_path=dataset_path)
        current_context += f"Question: {problem['question']}\n"
        current_context += f"Starter Code: {problem['starter_code']}\n"
        current_context += f"Code\n{context_json[str(index)]}"
        context.append(current_context)
    return "\n\n".join(context)

def create_system_prompt():
    return """
    **SYSTEM PROMPT**: 
    You are a CODE GENERATOR. You are asked to generate code for the following problem.
    YOU ONLY PROVIDE THE CODE!!!!!!!!!
    You need to generate the code that will pass the test cases.
    Context Prompt examples will be provided for Structure.
    Context Examples will have **FEW-SHOT EXAMPLE** Tag.

    You will provide code for the LAST Question.
    The question will have **USER PROMPT** Tag.
    After Code tag provide the PYTHON code only.

    GUIDELINES:
    - Write only the function implementation.
    - NO EXPLANATIONS
    - Ensure the code is complete, syntactically correct, and directly executable.
    - Do not include any tests, comments, or markdown code blocks.
    - Follow Python's PEP8 guidelines.
    - DO NOT add ```python and ```
    """

def generate_prompt(problem: dict) -> str:
    """
    Generate a prompt for a given problem.
    
    Args:
        problem (dict): A dictionary containing the problem, test cases, and starter code.
    
    Returns:
        str: A prompt for the given problem.
    """
    SYSTEM_PROMPT = create_system_prompt()
    CONTEXT_PROMPT = create_context_prompt(sample_size=5, dataset_path="./data/dev")
    
    prompt = SYSTEM_PROMPT + CONTEXT_PROMPT


    prompt += "\n\n**USER PROMPT**:\n"
    prompt += f"Question: {problem['question']}\n"
    prompt += f"Starter Code: {problem['starter_code']}\n"
    prompt += "\n\nCode\n"
    return prompt


def generate_code(problem: dict, client: Client) -> str:
    """
    Implement the generation function.
    
    Args:
        problem (dict): The problem to solve.
        
    Returns:
        str: The generated code.
    """
    
    prompt = generate_prompt(problem=problem)
    request = CompletionRequest(
        prompt=Prompt.from_text(prompt),
        maximum_tokens=512,
        temperature = 0,
    )
    # API reference for the client:
    # https://aleph-alpha-client.readthedocs.io/en/latest/
    response = client.complete(request, model=MODEL)
    
    # Run test cases on the generated code and iterate.
    # test_results = run_test_cases(
    #    problem=problem, 
    #    generation=response.completions[0].completion, 
    #    timeout=10,
    # )

    return response.completions[0].completion



problem = load_sample(index=6, dataset_path="./data/dev")
generated_code = generate_code(problem, client)


AttributeError: module 'signal' has no attribute 'SIGALRM'

In [ ]:
test_results = run_test_cases(
    problem=load_sample(index=6, dataset_path="./data/dev"), 
    generation=generated_code, 
    timeout=10,
)
print(test_results)

In [ ]:
from utilities import score

passed_problems, passed_test_cases = score(
    generation_func=generate_code, 
    client=client,
    dataset_path="./data/val", 
    length=5,
)

print(f"Passed {passed_problems*100}% of problems")
print(f"Passed {passed_test_cases*100}% of test cases")